
# 📊 Proyecto Flask + Google Sheets + Power BI

Paso a paso proyecto en **Python + Flask** que:
- Maneja inicio de sesión de usuarios
- Registra accesos en **Google Sheets**
- Muestra un **Dashboard de Power BI**
- Permite descargar un archivo Excel

El objetivo es **explicar el código por secciones**, facilitando su mantenimiento y aprendizaje.


## 📦 Importación de librerías

In [ ]:
# =========================
# IMPORTACIONES
# =========================
import os
import time
import json

import gspread
from google.oauth2.service_account import Credentials

from flask import (
    Flask,
    render_template,
    request,
    redirect,
    url_for,
    session,
    send_from_directory,
    send_file
)

from datetime import datetime, timedelta

## 🌎 Configuración de Zona Horaria

In [ ]:
# =========================
# ZONA HORARIA
# =========================
os.environ["TZ"] = "America/Mexico_City"

# tzset solo existe en Linux / macOS
if hasattr(time, "tzset"):
    time.tzset()


## ⚙️ Configuración de Flask

In [ ]:
# =========================
# CONFIGURACIÓN FLASK
# =========================
app = Flask(__name__)
app.secret_key = os.environ.get("SECRET_KEY", "keytronics123")
app.permanent_session_lifetime = timedelta(minutes=30)

## 📄 Configuración de Google Sheets

In [ ]:
# =========================
# GOOGLE SHEETS
# =========================
GOOGLE_SHEETS_ID = os.environ.get(
    "GOOGLE_SHEETS_ID",
    "1bifYmIaFzdenLR9MlVE_0rYpLWRQ-yjBpGQIEtiyZsk"
)

CREDENCIALES_JSON = "credenciales_google.json"
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

print("🔍 Intentando cargar credenciales (ENV o archivo)...")

## 🔐 Autenticación con Google Sheets

In [ ]:
sheet = None

try:
    google_credentials_env = os.environ.get("GOOGLE_CREDENTIALS")

    if google_credentials_env:
        # Render / Producción
        creds_info = json.loads(google_credentials_env)
        credentials = Credentials.from_service_account_info(
            creds_info,
            scopes=SCOPES
        )
        print("🔑 Credenciales cargadas desde variable de entorno.")
    else:
        # Desarrollo local
        credentials = Credentials.from_service_account_file(
            CREDENCIALES_JSON,
            scopes=SCOPES
        )
        print("🔑 Credenciales cargadas desde archivo local.")

    client = gspread.authorize(credentials)
    sheet = client.open_by_key(GOOGLE_SHEETS_ID).sheet1

    print("✅ Conexión exitosa con Google Sheets.")

except Exception as e:
    print("⚠️ Error conectando con Google Sheets:", e)
    sheet = None

## 👥 Usuarios permitidos y URL's

In [ ]:
# =========================
# USUARIOS PERMITIDOS
# =========================
USUARIOS = {
    "Jose_Consultor": {
        "password": "654321987",
        "roles": ["reporte_1", "reporte_2"]
    },

    "AlfonsoCampo": {
        "password":"N7$vL8qY#x3B",
        "roles": ["reporte_1", "reporte_2"]
    },

    "AlejandroCampo": {
        "password": "tR!5mK2wQ9#z",
        "roles": ["reporte_1"]
    },

    "DavidVargas": {
        "password":"Fp8#Vd4!sZ1q",
        "roles": ["reporte_1"]
    },

    "BrendaMuñoz": {
        "password":"H2!xW7qR#k9L",
        "roles": ["reporte_1"]
    },

    "VidalCamacho": {
        "password": "b9#Zt6Pq!M3r",
        "roles": ["reporte_1", "reporte_2"]
    },

    "HumbertoColin": {
        "password":"Gv3!Qw8#xN5z",
        "roles": ["reporte_1", "reporte_2"]
    },

    "JaimeFontanet": {
        "password":"L1#pT9v!R6kS",
        "roles": ["reporte_1", "reporte_2"]
    },

    "AlbertoEchavarria": {
        "password":"s4!Kz7Q#n2Wm",
        "roles": ["reporte_1", "reporte_2"]
    },

    "Laura": {
        "password":"Y8!fR5p#T1qZ",
        "roles": ["reporte_2"]
    }
}

POWER_BI_REPORTES = {
    "reporte_1": "https://app.powerbi.com/view?r=eyJrIjoiMjA2MWYwNTAtODk4NS00NzQ1LTg3MmItOTk1Y2Y0N2FiNmJhIiwidCI6IjAzODk5MTIxLWQ5NzYtNDRlOS1iODI0LTFmYzU1N2JmZGRjZSJ9",
    "reporte_2": "https://app.powerbi.com/view?r=eyJrIjoiYzQ4N2I4M2QtMWIxOC00MTdmLWI5MDItMjYwZWRkY2RkMTZhIiwidCI6IjAzODk5MTIxLWQ5NzYtNDRlOS1iODI0LTFmYzU1N2JmZGRjZSJ9"
}

## 📝 Función para registrar accesos

In [ ]:
# =========================
# FUNCIÓN: REGISTRAR ACCESO
# =========================
def registrar_acceso(usuario):
    print("🚨 registrar_acceso() llamado por:", usuario)

    if not sheet:
        print("⚠️ No hay conexión con Google Sheets.")
        return

    try:
        fecha = datetime.now().strftime("%Y-%m-%d")
        hora = datetime.now().strftime("%H:%M:%S")

        sheet.append_row([usuario, fecha, hora])
        print(f"✅ Acceso registrado: {usuario} - {fecha} {hora}")

    except Exception as e:
        print("❌ Error al guardar en Google Sheets:", e)


## 🔐 Rutas de Login

In [ ]:
# =========================
# RUTAS
# =========================
@app.route("/")
def home():
    return render_template("login.html")

@app.route("/login", methods=["POST"])
def login():
    u = request.form["username"]
    p = request.form["password"]

    if u in USUARIOS and USUARIOS[u]["password"] == p:
        session["user"] = u
        session["roles"] = USUARIOS[u]["roles"]
        registrar_acceso(u)
        return redirect("/dashboard")

    return render_template("login.html", error="Credenciales incorrectas")

## 📊 Dashboard con Power BI

In [ ]:
@app.route("/dashboard")
def dashboard():
    if "user" not in session:
        return redirect("/")
    return render_template("selector_reportes.html", roles=session["roles"])

@app.route("/reporte/<reporte_id>")
def ver_reporte(reporte_id):
    if "user" not in session:
        return redirect(url_for("home"))

    if reporte_id not in POWER_BI_REPORTES:
        return "Reporte no existe", 404

    if reporte_id not in session.get("roles", []):
        return "⛔ Sin permiso", 403

    return render_template(
        "dashboard.html",
        embed_url=POWER_BI_REPORTES[reporte_id],
        reporte_id=reporte_id
    )

## 📥 Descarga de archivo

In [ ]:
@app.route("/descargar_csv")
def descargar_csv():
    if "user" not in session:
        return redirect(url_for("home"))

    return send_from_directory(
        os.getcwd(),
        "datos_reporte_Keytronics.xlsx",
        as_attachment=True
    )

@app.route("/descargar_inventario")
def descargar_inventario():
    if "user" not in session:
        return redirect(url_for("home"))

    return send_from_directory(
        os.getcwd(),
        "Reporte_inventario.xlsx",
        as_attachment=True
    )

## 🚪 Logout

In [ ]:
@app.route("/logout")
def logout():
    session.clear()
    return redirect("/")

## ▶️ Ejecución del servidor

In [ ]:
# =========================
# EJECUCIÓN
# =========================
if __name__ == "__main__":
    port = int(os.environ.get("PORT", 10000))
    print("🚀 Servidor iniciando en puerto:", port)
    app.run(host="0.0.0.0", port=port, debug=False)